In [8]:
import logging
import sys

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

from smatch import smatch_util
from smatch import smatch_amr
from AMRGraph import AMR
import AMRData
from amr_util.demo_util import get_custom_amr
from amr_util.demo_util import get_smatch
from postprocessing import ActionSequenceReconstruction as asr
from preprocessing.action_sequence_generators.simple_asg import SimpleASG
from preprocessing.action_sequence_generators.simple_asg__informed_swap import SimpleInformedSwapASG
from preprocessing.action_sequence_generators.simple_informed_break_nodes_on_stack import SimpleInformedWithBreakNodesOnStackASG
from preprocessing.action_sequence_generators.backtracking_asg import BacktrackingASGFixedReduce

# Action Sequence Generation: initial behaviour

## Smatch = 1 

The sentence and its AMR representation are:

In [4]:
# ::id bolt12_10489_5932.9 ::amr-annotator SDL-AMR-09 ::preferred
sentence="In addition , there was something else that was very suspicious ."
amr_str = """(a / and~e.0,1 
      :op2 (s / suspicious~e.10 
            :domain~e.4,8 (s3 / something~e.5 
                  :mod (e / else~e.6)) 
            :degree (v / very~e.9)))"""

Storing the data in the customized amr structure:

In [5]:
custom_amr = get_custom_amr(amr_str)

Generating the action sequence using the initial version of the algorithm

In [6]:
asg_implementation = SimpleASG(1, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)

The obtained action sequence:

In [7]:
for action in action_sequence:
    print(action.to_string())

SH_and
DN
DN
DN
DN
SH_something
SH_else
RR_mod
DN
DN
SH_very
SH_suspicious
RL_degree
RL_domain
RR_op2
DN


AMR obtained by applying action sequence:

In [8]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)

( d1 / and 
	:op2  ( d1_1 / suspicious 
		:degree  ( d1_1_1 / very )
		:domain  ( d1_1_2 / something 
			:mod  ( d1_1_2_1 / else )
		)
	)
)


Smatch obtained for this example:

In [9]:
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

1.0


## Smatch < 1 

An example follows where the action sequence found leads to a subgraph of the initial AMR. The sentence and its AMR representation are:

In [14]:
sentence="Facing the caprice of nature , humankind is really insignificant ."
amr_str="""(s / significant-02~e.9 :polarity~e.9 -~e.9 
      :ARG1 (h / humankind~e.6) 
      :degree (r / really~e.8) 
      :condition (f / face-01~e.0 
            :ARG0 h 
            :ARG1 (c / caprice~e.2 
                  :poss~e.3 (n / nature~e.4))))"""

In [15]:
custom_amr = get_custom_amr(amr_str)

In [16]:
asg_implementation = SimpleASG(1, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)

In [17]:
for action in action_sequence:
    print(action.to_string())

SH_face-01
DN
SH_caprice
DN
SH_nature
RR_poss
RR_ARG1
DN
SH_humankind
RR_ARG0
DN
SH_really
SH_significant-02
RL_degree
RL_condition
DN


AMR obtained by applying action sequence:

In [18]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)

( d1 / significant-02 
	:degree  ( d1_1 / really )
	:condition  ( d1_2 / face-01 
		:ARG1  ( d1_2_1 / caprice 
			:poss  ( d1_2_1_1 / nature )
		)
		:ARG0  ( d1_2_2 / humankind )
	)
)


Smatch obtained for this example:

In [19]:
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

0.923076923077


# Action Sequence Generation: Informed Swap

For the examples bellow, the initial algorithm version fails, but the version build using the Informed Swap strategy (perform a swap action when it leads to a reduce) finds an action sequence.

In [55]:
sentence = "A funny story ( at least to me ) ."
amr_str = """(s / story~e.2 
      :mod (f / funny~e.1 
            :prep-to~e.6 (i / i~e.7) 
            :mod (a / at-least~e.4,5)))"""
custom_amr = get_custom_amr(amr_str)

Using the Informed Swap action sequence generation implementation, with Swap of depth 1 and no Rotate action:

In [57]:
asg_implementation = SimpleInformedSwapASG(1, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
for action in action_sequence:
    print(action.to_string())

DN
SH_funny
SH_story
DN
SH_at-least
SW
RR_mod
DN
DN
SH_i
RR_prep-to
RR_mod
DN
DN


Generated AMR and smatch score:

In [23]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

( d1 / story 
	:mod  ( d1_1 / funny 
		:mod  ( d1_1_1 / at-least )
		:prep-to  ( d1_1_2 / i )
	)
)
1.0


Let me take another example, which contains coreference, but an action sequence is still found (which leds to a subgraph of the initial AMR)

In [60]:
sentence = "Do many other guys choose to moisturize ?"
amr_str = """(c / choose-01~e.4 :mode~e.7 interrogative~e.7 
      :ARG0 (g / guy~e.3 
            :mod (o / other~e.2) 
            :quant (m / many~e.1)) 
      :ARG1 (m2 / moisturize-01~e.6 
            :ARG0 g))"""
custom_amr = get_custom_amr(amr_str)
asg_implementation = SimpleInformedSwapASG(1, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
for action in action_sequence:
    print(action.to_string())

DN
SH_many
SH_other
SH_guy
RL_mod
RL_quant
SH_choose-01
DN
SH_moisturize-01
SW
RL_ARG0
RR_ARG1
SH_interrogative
RR_mode


In [28]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

( d1 / choose-01 
	:ARG1  ( d1_1 / moisturize-01 
		:ARG0  ( d1_1_1 / guy 
			:mod  ( d1_1_1_1 / other )
			:quant  ( d1_1_1_2 / many )
		)
	)
	:mode interrogative
)
0.95652173913


# Action Sequence Generation: Informed Swap (Integrating Swap<sub>2</sub>)

In [31]:
sentence = "I am not saying that women should realistically be able to dance down back alleys naked ."
amr_str = """(s / say-01~e.3 :polarity~e.2 -~e.2 
      :ARG0 (i / i~e.0) 
      :ARG1~e.4 (r / recommend-01~e.6 
            :ARG1 (p / possible-01~e.9 
                  :ARG1 (d / dance-01~e.11 
                        :ARG0 (w / woman~e.5) 
                        :manner (n / naked~e.15) 
                        :location (a / alley~e.14 
                              :mod (b2 / back~e.13)) 
                        :direction (d2 / down~e.12)) 
                  :manner (r2 / realistic-03~e.7))))"""
custom_amr = get_custom_amr(amr_str)

Note that the first parameter is now 2 (to indicate that Swap<sub>2</sub> should be integrated):

In [32]:
asg_implementation = SimpleInformedSwapASG(2, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
for action in action_sequence:
    print(action.to_string())

SH_i
DN
SH_-
SH_say-01
RL_polarity
RL_ARG0
DN
SH_woman
SH_recommend-01
SH_realistic-03
DN
SH_possible-01
RL_manner
DN
SH_dance-01
SW_2
RL_ARG0
SH_down
RR_direction
SH_back
SH_alley
RL_mod
RR_location
SH_naked
RR_manner
SW
RR_ARG1
RR_ARG1
RR_ARG1
DN


In [33]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

( d1 / say-01 
	:polarity -
	:ARG0  ( d1_1 / i )
	:ARG1  ( d1_2 / recommend-01 
		:ARG1  ( d1_2_1 / possible-01 
			:manner  ( d1_2_1_1 / realistic-03 )
			:ARG1  ( d1_2_1_2 / dance-01 
				:ARG0  ( d1_2_1_2_1 / woman )
				:direction  ( d1_2_1_2_2 / down )
				:location  ( d1_2_1_2_3 / alley 
					:mod  ( d1_2_1_2_3_1 / back )
				)
				:manner  ( d1_2_1_2_4 / naked )
			)
		)
	)
)
1.0


Let's take another example:

In [24]:
sentence = "He should n't be expected to be anything ."
amr_str = """(r / recommend-01~e.1 
      :ARG1 (e / expect-01~e.4 :polarity~e.2 -~e.2 
            :ARG1 (a / anything~e.7 
                  :domain~e.3,6 (h / he~e.0))))"""
custom_amr = get_custom_amr(amr_str)
asg_implementation = SimpleInformedSwapASG(3, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
for action in action_sequence:
    print(action.to_string())
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

SH_he
SH_recommend-01
SH_-
DN
SH_expect-01
RL_polarity
DN
DN
SH_anything
SW_2
RL_domain
SW
RR_ARG1
RR_ARG1
DN
( d1 / recommend-01 
	:ARG1  ( d1_1 / expect-01 
		:polarity -
		:ARG1  ( d1_1_1 / anything 
			:domain  ( d1_1_1_1 / he )
		)
	)
)
1.0


# Action Sequence Generation: Informed Swap (Integrating Swap<sub>3</sub>)

In [34]:
sentence = "It seems though that any old sod can do it ."
amr_str = """(h / have-concession-91~e.2 
      :ARG1 (s / seem-01~e.1 
            :ARG1 (p / possible-01~e.7 
                  :ARG1 (d / do-02~e.8 
                        :ARG0 (s2 / sod~e.6 
                              :mod (a / any~e.4) 
                              :mod (o / old~e.5)) 
                        :ARG1 (i / it~e.0,9)))))"""
custom_amr = get_custom_amr(amr_str)

In [36]:
asg_implementation = SimpleInformedSwapASG(3, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
for action in action_sequence:
    print(action.to_string())

SH_it
SH_seem-01
SH_have-concession-91
DN
SH_any
SH_old
SH_sod
RL_mod
RL_mod
SH_possible-01
SH_do-02
SW
RL_ARG0
SW_3
RL_ARG1
SW_2
RR_ARG1
RR_ARG1
RR_ARG1
DN
DN


In [37]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

( d1 / have-concession-91 
	:ARG1  ( d1_1 / seem-01 
		:ARG1  ( d1_1_1 / possible-01 
			:ARG1  ( d1_1_1_1 / do-02 
				:ARG0  ( d1_1_1_1_1 / sod 
					:mod  ( d1_1_1_1_1_1 / old )
					:mod  ( d1_1_1_1_1_2 / any )
				)
				:ARG1  ( d1_1_1_1_2 / it )
			)
		)
	)
)
1.0


# Action Sequence Generation: Informed Swap (Integrating Rotate)

In [40]:
sentence = "This is ultimately why your body may produce some version of these chemicals naturally ."
amr_str = """(c / cause-01~e.3 
      :ARG1 (p / possible-01~e.6 
            :ARG1 (p2 / produce-01~e.7 
                  :ARG0 (b / body~e.5 
                        :poss~e.4 (y / you~e.4)) 
                  :ARG1 (v / version~e.9 
                        :quant (s / some~e.8) 
                        :mod (c2 / chemical~e.12 
                              :mod (t / this~e.0,11))) 
                  :ARG1-of (n / natural-03~e.13))) 
      :mod (u / ultimate~e.2))"""
custom_amr = get_custom_amr(amr_str)

Notice how the second parameter was changed to "True":

In [42]:
asg_implementation = SimpleInformedSwapASG(3, True)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
for action in action_sequence:
    print(action.to_string())

SH_this
DN
SH_ultimate
SH_cause-01
RL_mod
SH_you
SH_body
RL_poss
SH_possible-01
SH_produce-01
SW
RL_ARG0
SH_some
SH_version
RL_quant
DN
DN
SH_chemical
RO
RL_mod
SW_3
RR_mod
SW_2
RR_ARG1
SH_natural-03
RR_ARG1-of
SW
RR_ARG1
RR_ARG1
DN


In [44]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

( d1 / cause-01 
	:mod  ( d1_1 / ultimate )
	:ARG1  ( d1_2 / possible-01 
		:ARG1  ( d1_2_1 / produce-01 
			:ARG0  ( d1_2_1_1 / body 
				:poss  ( d1_2_1_1_1 / you )
			)
			:ARG1  ( d1_2_1_2 / version 
				:quant  ( d1_2_1_2_1 / some )
				:mod  ( d1_2_1_2_2 / chemical 
					:mod  ( d1_2_1_2_2_1 / this )
				)
			)
			:ARG1-of  ( d1_2_1_3 / natural-03 )
		)
	)
)
1.0


# Action Sequence Generation: Informed Swap (Integrating Break-token)

In [ ]:
sentence = "Middle school buildings collapsed , and young students were buried under the crumbled walls ."
amr_str = """(a / and~e.5 
      :op1 (c / collapse-01~e.3 
            :ARG1 (b2 / building~e.2 
                  :mod (s / school~e.1 
                        :mod (m / middle~e.0)))) 
      :op2 (b / bury-01~e.9 
            :ARG1 (p / person~e.7 
                  :mod (y / young~e.6) 
                  :ARG0-of~e.7 (s2 / study-01~e.7)) 
            :ARG2 (u / under~e.10 
                  :op1 (w / wall~e.13 
                        :ARG1-of (c2 / crumble-01~e.12)))))"""
custom_amr = get_custom_amr(amr_str)

In [46]:
asg_implementation = SimpleInformedWithBreakNodesOnStackASG(1, False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
for action in action_sequence:
    print(action.to_string())

SH_middle
SH_school
RL_mod
SH_building
RL_mod
SH_collapse-01
RL_ARG1
DN
SH_and
RL_op1
SH_young
BRK_study-01_person
RL_ARG0-of
RL_mod
DN
SH_bury-01
RL_ARG1
SH_under
DN
SH_crumble-01
SH_wall
RL_ARG1-of
RR_op1
RR_ARG2
RR_op2
DN


In [47]:
generated_amr_str = asr.reconstruct_all(action_sequence)
print(generated_amr_str)
smatch_f_score = get_smatch(generated_amr_str,amr_str)
print(smatch_f_score)

( d1 / and 
	:op1  ( d1_1 / collapse-01 
		:ARG1  ( d1_1_1 / building 
			:mod  ( d1_1_1_1 / school 
				:mod  ( d1_1_1_1_1 / middle )
			)
		)
	)
	:op2  ( d1_2 / bury-01 
		:ARG1  ( d1_2_1 / person 
			:ARG0-of  ( d1_2_1_1 / study-01 )
			:mod  ( d1_2_1_2 / young )
		)
		:ARG2  ( d1_2_2 / under 
			:op1  ( d1_2_2_1 / wall 
				:ARG1-of  ( d1_2_2_1_1 / crumble-01 )
			)
		)
	)
)
1.0


# Action Sequence Generation: Local Backtracking

The example bellow cannot be processed using the Informed Swap strategy with the initial action set. However, it can be processed when integrating Swap<sub>2</sub>. Yet, an action sequence constructed using only the actions from the initial set exists and it is found by backtracking (the found action sequence contains an ordering of Swap and Shift actions that actually solves the non-projectivity, but this ordering is not reached by the Informed Swap strategy as the Swap action does not lead immediately to a reduce).

In [19]:
sentence = "He should n't be expected to be anything ."
amr_str = """(r / recommend-01~e.1 
      :ARG1 (e / expect-01~e.4 :polarity~e.2 -~e.2 
            :ARG1 (a / anything~e.7 
                  :domain~e.3,6 (h / he~e.0))))"""
custom_amr = get_custom_amr(amr_str)

In [22]:
max_depth = 4 * len(sentence)
asg_implementation = BacktrackingASGFixedReduce(1,max_depth)
#asg_implementation = SimpleInformedSwapASG(2,False)
action_sequence = asg_implementation.generate_action_sequence(custom_amr, sentence)
generated_amr_str = asr.reconstruct_all(action_sequence)
smatch_f_score = get_smatch(generated_amr_str, amr_str)
for action in action_sequence:
    print(action.to_string())
print(smatch_f_score)

SH_he
SH_recommend-01
SH_-
DN
SH_expect-01
RL_polarity
DN
DN
SW
SH_anything
DN
SW
RL_domain
RR_ARG1
RR_ARG1
1.0
